In [1]:
import numpy as np
from scipy import stats

X_test = np.load('/content/drive/MyDrive/Skóli/3V/Gervigreind/Lokaverkefni/time_dependant_fylki/x_test_file.npy')
y_test = np.load('/content/drive/MyDrive/Skóli/3V/Gervigreind/Lokaverkefni/time_dependant_fylki/y_test_file.npy')
X_train = np.load('/content/drive/MyDrive/Skóli/3V/Gervigreind/Lokaverkefni/time_dependant_fylki/x_train_file.npy')
y_train = np.load('/content/drive/MyDrive/Skóli/3V/Gervigreind/Lokaverkefni/time_dependant_fylki/y_train_file.npy')
X_val = np.load('/content/drive/MyDrive/Skóli/3V/Gervigreind/Lokaverkefni/time_dependant_fylki/x_val_file.npy')
y_val = np.load('/content/drive/MyDrive/Skóli/3V/Gervigreind/Lokaverkefni/time_dependant_fylki/y_val_file.npy')

In [2]:
def reduced(X):
    heildin = np.zeros([X.shape[0], 140])
    for i in range(X.shape[0]):
        f = X[i]

        medal = np.mean(f, axis=0)
        stadal = np.std(f, axis=0)
        skewid = stats.skew(f, axis=0)
        kurtosisinn = stats.kurtosis(f, axis=0)
        medianinn = np.median(f, axis=0)
        minid = np.min(f, axis=0)
        maxid = np.max(f, axis=0)

        fylki = np.append(medal, stadal)
        fylki = np.append(fylki, skewid)
        fylki = np.append(fylki, kurtosisinn)
        fylki = np.append(fylki, medianinn)
        fylki = np.append(fylki, minid)
        fylki = np.append(fylki, maxid)
        heildin[i,:] = fylki
    return heildin

In [3]:
X_train_nytt = reduced(X_train)
X_test_nytt = reduced(X_test)
X_val_nytt = reduced(X_val)

In [4]:
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer, StandardScaler
scaler = StandardScaler(copy=False)
scaler.fit_transform(X_train_nytt)
scaler.transform(X_val_nytt)
scaler.transform(X_test_nytt)


import tensorflow.keras as keras

model = keras.Sequential([


    # 1st dense layer
    keras.layers.Dense(128, input_dim=X_train_nytt.shape[1],activation='relu', kernel_regularizer = keras.regularizers.l2(0.0002)),
    keras.layers.Dropout(0.3),

    # 2nd dense layer
    keras.layers.Dense(64, activation='relu', kernel_regularizer = keras.regularizers.l2(0.0002)),
    keras.layers.Dropout(0.3),

    # 3rd dense layer
    keras.layers.Dense(32, activation='relu', kernel_regularizer = keras.regularizers.l2(0.0002)),
    keras.layers.Dropout(0.3),

    # output layer
    keras.layers.Dense(8, activation='softmax')
])

# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# train model
history = model.fit(X_train_nytt, y_train, validation_data=(X_val_nytt, y_val), batch_size=32, epochs=130)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               18048     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 2

In [5]:
test_loss, test_acc = model.evaluate(X_test_nytt, y_test, verbose=0)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

Test accuracy: 0.36672279238700867
Test loss: 1.7669527530670166
